# 01_benchmarking

This notebook compares the baseline performance of 12 algorithms on the SEO-Effect dataset. The performance is evaluated by measuring following evaluation metrics: Accuracy, ROC AUC (One-vs-One), Macro Precision, Macro Recall, Macro F1, as well as Fit and Train Time.

The algorithms are selected from the ["scikit-learn algorithm cheat-sheet"](https://scikit-learn.org/stable/_static/ml_map.png) to compare a broad spectrum of differnt classification methods.

Custom parameters are kept to a minimum. max_iteration is limited to 100, max_depth to 10, n_neighbors are 4, while the outlier_label is set to 5. The penalty option is set to l2 and the random state is set to 22. This is done to limit run time or because a parameter needs to be set.

The algorithms are cross validated by splitting the data into five stratified shuffle splits, with a test size of 66%. The mean of each metric as well the standard deviation is saved to a dataframe and stored in <code>output/01_benchmarking.csv</code>.

Six of the 12 algorithms perform very well, with an accuracy of at least 90% and an f1 of at least 75%. Those are (in order of best performance to last): GradientBoosting, RandomForest, ExtraTrees, DecisionTree, LinearSVC and GaussianNB. However the fit and training times of the GradientBoostingClassifier is 750 times longer than that of the Gaussian Naive Bayes Classifier. This classifier is known as a quick and reliable way to test classification performance and works well with the data that is being used. As such it will be used to compare data preprocessing methods in the next section.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_validate, StratifiedShuffleSplit
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings('ignore')

from IPython.display import clear_output

# load data
df = pd.read_csv('output/data_cleaned.csv')

# split data into features and target
X = df.drop(columns=['seo class'])
y = df['seo class']

# list of classifiers to compare
classifiers = {'AdaBoost': AdaBoostClassifier(),
               'BernoulliNB': BernoulliNB(),
               'DecisionTree': DecisionTreeClassifier(),
               'ExtraTrees': ExtraTreesClassifier(),
               'GaussianNB': GaussianNB(),
               'GradientBoosting': GradientBoostingClassifier(),
               'KNeighbors': KNeighborsClassifier(),
               'LinearSVC': LinearSVC(),
               'RadiusNeighbors': RadiusNeighborsClassifier(),
               'RandomForest': RandomForestClassifier(),
               'SGD': SGDClassifier(),
               'SVC': SVC()}

# dictionary of evaluation metrics
metrics = {'accuracy': 'accuracy',
           'precision': 'precision_macro', 
           'recall': 'recall_macro',
           'f1': 'f1_macro'}

# set minimal parameters to make sure algorithms function
params = {'max_iter': 100,
          'max_depth' : 10,
          'random_state': 22,
          'penalty': 'l2',
          'n_neighbors': 4,
          'outlier_label': 5}

# create stratified split for cross validation
sss = StratifiedShuffleSplit(n_splits=5, test_size=.66, random_state=22)

# empty dictionary to store results
cv_results = {}

# iterate over classifiers to compare results
for name, clf in classifiers.items():
    clear_output()
    print('Current classifier: %s' % (name))
    
    # get parameter options for current classifier
    clf_params = clf.get_params()
    
    # select matching parameters for current classifier from params
    c_params = {}
    for p in params.keys():
        if p in clf_params.keys():
            c_params[p] = params[p]
    
    # set parameters
    if c_params:
        clf.set_params(**c_params)
    
    # cross validate classifier
    cv = cross_validate(clf, X, y, scoring=metrics, cv=sss)
    # save results of cross validation
    cv_results[name] = cv
    
    # print f1/accuracy mean and std to update on results
    print('F1: %.2f±%.2f' % (cv['test_f1'].mean()*100, cv['test_f1'].std()*100))
    print('Acc: %.2f±%.2f' % (cv['test_accuracy'].mean()*100, cv['test_accuracy'].std()*100))
    
# format data for dataframe
data = []
for name, results in cv_results.items():
    row = [name]
    for k, v in results.items():
        # add mean and standard deviation to data
        row.append(v.mean())
        row.append(v.std())
    data.append(row)
    
# column names for dataframe
columns = ['classifier']
for k in cv.keys():
    k = k.replace('test_', '')
    columns.append(k+'_mean')
    columns.append(k+'_std')

# create data frame to display cv results
results = pd.DataFrame(data, columns=columns)
# save data frame as csv file
results.to_csv('output/01_benchmarking.csv')

Current classifier: GaussianNB
F1: 75.85±0.13
Acc: 96.33±0.14


In [2]:
# load results from csv
# (used to avoid having to run the whole code again)
import pandas as pd
r_df = pd.read_csv('output/01_benchmarking.csv', index_col=0)

# sort by best f1 score
r_df.sort_values(by=['f1_mean'], ascending=False)

,classifier,fit_time_mean,fit_time_std,score_time_mean,score_time_std,accuracy_mean,accuracy_std,precision_mean,precision_std,recall_mean,recall_std,f1_mean,f1_std
5,GradientBoosting,293.888358,19.446112,3.783617,0.084839,0.999940,0.000016,0.992321,0.001307,0.997442,0.001657,0.994853,0.001355
2,DecisionTree,0.764748,0.026987,0.343171,0.009292,0.998987,0.000186,0.919814,0.022215,0.893322,0.007192,0.905148,0.013507
3,ExtraTrees,5.674922,0.061941,2.473807,0.009516,0.990185,0.004118,0.961476,0.005121,0.794766,0.003457,0.824828,0.004186
9,RandomForest,11.830696,0.112538,2.736282,0.017617,0.997443,0.000419,0.987984,0.007488,0.791389,0.002575,0.821753,0.003601
7,LinearSVC,14.838831,0.308020,0.336247,0.002850,0.976518,0.038317,0.796508,0.053693,0.774725,0.043862,0.775224,0.031896
4,GaussianNB,0.399622,0.016154,0.713168,0.020434,0.963281,0.001391,0.757487,0.000663,0.970410,0.001742,0.758505,0.001312
1,BernoulliNB,0.376738,0.010776,0.549465,0.015261,0.916452,0.001131,0.733035,0.003055,0.939055,0.003086,0.734415,0.001984
0,AdaBoost,9.074909,0.079397,3.082461,0.049708,0.902365,0.143389,0.717554,0.039426,0.697937,0.081537,0.689125,0.095936
6,KNeighbors,0.284773,0.055302,945.714385,26.668485,0.726202,0.000711,0.589559,0.008776,0.535577,0.002654,0.545939,0.001965
10,SGD,5.137766,0.936832,0.334540,0.000846,0.904352,0.021051,0.523067,0.040010,0.514675,0.059501,0.504511,0.028937


In [3]:
# get standard dev columns to remove from df display
std_c = [c for c in r_df.columns if '_std' in c]

# set filters to narrow down results
# Filters: F1 > 75% and Accuracy > 95%
filter_ = (r_df['f1_mean'] > 0.75) & (r_df['accuracy_mean'] > 0.95)

# filter results by f1 > 75% and accuracy > 95%, sort by fit time
r_df[filter_].sort_values(by=['fit_time_mean']).drop(columns=std_c)

,classifier,fit_time_mean,score_time_mean,accuracy_mean,precision_mean,recall_mean,f1_mean
4,GaussianNB,0.399622,0.713168,0.963281,0.757487,0.970410,0.758505
2,DecisionTree,0.764748,0.343171,0.998987,0.919814,0.893322,0.905148
3,ExtraTrees,5.674922,2.473807,0.990185,0.961476,0.794766,0.824828
9,RandomForest,11.830696,2.736282,0.997443,0.987984,0.791389,0.821753
7,LinearSVC,14.838831,0.336247,0.976518,0.796508,0.774725,0.775224
5,GradientBoosting,293.888358,3.783617,0.999940,0.992321,0.997442,0.994853
